### Build a Sequential Neural Network using the familiar Census Data

In [5]:
# From last week
import keras
import tensorflow as tf
tf.__version__

'2.12.0'

### 1. Import the now (very) familiar US Census Income Data

In [6]:
import pandas
from sklearn.svm import SVC
import sklearn
from sklearn import preprocessing

from sklearn.model_selection import train_test_split

col_names = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

df = pandas.read_csv(r'C:\Users\khalil\ML Lab 11\adult.csv')
df.columns=col_names
df['Income'] = df['Income'].apply(lambda x: 0 if x == ' <=50K' else 1)

### 2. Split the data and use preprocessing StandardScaler on the data

In [10]:

for column in df.columns:
    if df[column].dtype == type(object):
        le = sklearn.preprocessing.LabelEncoder()
        df[column] = le.fit_transform(df[column])
df.tail()

,Age,WorkClass,fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
48837,27,4,257302,7,12,2,13,5,4,0,0,0,38,39,1
48838,40,4,154374,11,9,2,7,0,4,1,0,0,40,39,1
48839,58,4,151910,11,9,6,1,4,4,0,0,0,40,39,1
48840,22,4,201490,11,9,4,1,3,4,1,0,0,20,39,1
48841,52,5,287927,11,9,2,4,5,4,0,15024,0,40,39,1


In [11]:
X = df.drop(columns='Income')
X = pandas.DataFrame(preprocessing.StandardScaler().fit_transform(X), columns=X.columns)
y = df['Income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=555, stratify=y)

### 3. Use the Tensorflow and Keras libraries from last week to build a NN.

*Use today's class notes as a guild.  Try using different activation functions

In [12]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [16]:
nn_X_train = np.reshape(X_train.values, (X_train.shape[0], X_train.shape[1], 1))
nn_X_test = np.reshape(X_test.values, (X_test.shape[0], X_test.shape[1], 1))

In [17]:
mdl_k = tf.keras.Sequential([tf.keras.layers.Input(shape=(nn_X_train.shape[1], nn_X_train.shape[2]))])
mdl_k.add(tf.keras.layers.Dense(12, input_dim=3, activation='relu', kernel_initializer="he_normal"))
mdl_k.add(tf.keras.layers.Dropout(0.5))
mdl_k.add(tf.keras.layers.Dense(8, activation='relu'))

mdl_k.add(tf.keras.layers.LSTM(64))
mdl_k.add(tf.keras.layers.LeakyReLU(alpha=0.2))
mdl_k.add(tf.keras.layers.Dense(1, activation='relu'))

In [18]:
# Compile model
mdl_k.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

In [19]:
mdl_k.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 14, 12)            24        
                                                                 
 dropout (Dropout)           (None, 14, 12)            0         
                                                                 
 dense_1 (Dense)             (None, 14, 8)             104       
                                                                 
 lstm (LSTM)                 (None, 64)                18688     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 18,881
Trainable params: 18,881
Non-trai

In [20]:
val_dataset = tf.data.Dataset.from_tensor_slices((nn_X_test, y_test)).batch(batch_size=128)

In [21]:
%%time
history = mdl_k.fit(
    nn_X_train,
    y_train,
    epochs=50,
    batch_size=128,
    # validation_data=(nn_X_test, y_test),
    validation_data=val_dataset,
    #validation_split=0.1,
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", verbose=2)
#     ],
    use_multiprocessing=True
)

Epoch 1/50
306/306 [==============================] - 13s 25ms/step - loss: 0.0311 - accuracy: 0.9661 - val_loss: 1.5741e-04 - val_accuracy: 1.0000
Epoch 2/50
306/306 [==============================] - 6s 21ms/step - loss: 1.1292e-04 - accuracy: 1.0000 - val_loss: 3.6745e-05 - val_accuracy: 1.0000
Epoch 3/50
306/306 [==============================] - 7s 24ms/step - loss: 5.1590e-05 - accuracy: 1.0000 - val_loss: 1.3070e-05 - val_accuracy: 1.0000
Epoch 4/50
306/306 [==============================] - 7s 22ms/step - loss: 2.9594e-05 - accuracy: 1.0000 - val_loss: 1.1102e-05 - val_accuracy: 1.0000
Epoch 5/50
306/306 [==============================] - 7s 24ms/step - loss: 2.0050e-05 - accuracy: 1.0000 - val_loss: 7.5036e-06 - val_accuracy: 1.0000
Epoch 6/50
306/306 [==============================] - 7s 23ms/step - loss: 1.5214e-05 - accuracy: 1.0000 - val_loss: 5.8028e-06 - val_accuracy: 1.0000
Epoch 7/50
306/306 [==============================] - 7s 21ms/step - loss: 1.0812e-05 - accuracy:

### 4. Build an xgBoost model on this data.  Were you able to build a NN with greater accuracy than xgBoost ?

In [22]:
import statsmodels.api as sm
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [23]:
col_names = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

df = pandas.read_csv(r'C:\Users\khalil\ML Lab 9\adult.data')
df.columns=col_names
df['Income'] = df['Income'].apply(lambda x: 0 if x == ' <=50K' else 1)

for column in df.columns:
    if df[column].dtype == type(object):
        le = sklearn.preprocessing.LabelEncoder()
        df[column] = le.fit_transform(df[column])
df.tail()

,Age,WorkClass,fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
32555,27,4,257302,7,12,2,13,5,4,0,0,0,38,39,0
32556,40,4,154374,11,9,2,7,0,4,1,0,0,40,39,1
32557,58,4,151910,11,9,6,1,4,4,0,0,0,40,39,0
32558,22,4,201490,11,9,4,1,3,4,1,0,0,20,39,0
32559,52,5,287927,11,9,2,4,5,4,0,15024,0,40,39,1


In [24]:
X = df.drop(columns='Income')
y = df['Income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=555, stratify=y)

In [25]:
%%time
clf_XGB = XGBClassifier(n_estimators = 100, seed=555, use_label_encoder=False, eval_metric='logloss')
clfs = []
print('5-fold cross validation:\n')
for clf, label in zip([clf_XGB], ['XGBoost']):
    scores = sklearn.model_selection.cross_val_score(clf, X_train, y_train, cv=5, scoring="accuracy")
    print("Train CV Accuracy: %0.3f (+/- %0.3f) [%s]" % (scores.mean(), scores.std(), label))
    md = clf.fit(X_train, y_train)
    clfs.append(md)
    print("Test Accuracy: %0.4f " % (sklearn.metrics.accuracy_score(clf.predict(X_test), y_test)))

C:\Users\khalil\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\khalil\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


5-fold cross validation:



C:\Users\khalil\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\khalil\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\khalil\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\khalil\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Train CV Accuracy: 0.866 (+/- 0.002) [XGBoost]
Test Accuracy: 0.8709 
Wall time: 10.6 s


Yeah, NN Accuracy (100%) and it is Greater than XGBoost Classifier Accuracy(87%)